In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import re
sys.path.append('../')
from common import *
from tqdm import tqdm
import multiprocessing
from functools import partial
tqdm.pandas()
import numpy as np

In [4]:
###input
cut_off = 0.3
num = 'all'
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'
cut_off_remain_path = f'../../../Results/lipid/top{num}_{cut_off}_remain_re/'
not_lipid_YMDB_success_met_smile_add_no_ec_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/YMDB_success_met_smile_uptake_top50_0.3.pickle'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
rxndb_to_model_add_no_ec_path = cut_off_remain_path + f'rxndb_to_model_top{num}_{cut_off}_remain.csv'
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'

# lipid_YMDB_success_met_smile_file_remian ='../../../Results/lipid/topall_0.3_remain/YMDB_success_met_smile_topall_0.3_remain.pickle'
lipid_YMDB_success_met_smile_file_remian ='../../../Results/lipid/topall_0.3_remain_re/YMDB_success_met_smile_topall_0.3_remain.pickle'
# lipid_YMDB_success_met_smile_file = '../../../Results/lipid/topall_0.3/YMDB_success_met_smile_topall_0.3.pickle'
lipid_YMDB_success_met_smile_file = '../../../Results/lipid/topall_0.3_re/YMDB_success_met_smile_topall_0.3.pickle'
###output
rxndb_met_max_score_output_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_merge.csv'
rxndb_to_model_with_target_path = cut_off_path + f'rxndb_to_model_with_target_topall_0.3.csv'
rxndb_to_model_with_target_success_path = cut_off_path + 'rxndb_to_model_with_target_topall_0.3_success.csv'

In [5]:
def get_score_from_smiles(input_smiles,smile_max_score_pd): ###return highest similarity score
    # input_smiles = normalize_smiles(input_smiles)
    row = smile_max_score_pd[smile_max_score_pd['inchikey0'] == smiles2inchikey0(input_smiles)]
    if not row.empty:
        return row['score'].max()
    else:
        return 0
def process_reaction_product(index_row,smile_max_score_pd):
    index, row = index_row
    reactant_smiles = row['rxn_smiles_basic'].split('>>')[0].split('.')
    product_smiles = row['rxn_smiles_basic'].split('>>')[1].split('.')
    scores = []

    for i in product_smiles:
        scores.append(get_score_from_smiles(i, smile_max_score_pd))

    scores_all_0 = all(score == 1 for score in scores)

    if scores_all_0 and len(reactant_smiles)>0:
        return reactant_smiles, row['NO']
    else:
        return [], None
def process_reaction_reactant(index_row,smile_max_score_pd):
    index, row = index_row
    reactant_smiles = row['rxn_smiles_basic'].split('>>')[0].split('.')
    product_smiles = row['rxn_smiles_basic'].split('>>')[1].split('.')
    scores = []

    for i in reactant_smiles:
        scores.append(get_score_from_smiles(i, smile_max_score_pd))

    scores_all_0 = all(score == 1 for score in scores)

    if scores_all_0 and len(product_smiles)>0:
        return product_smiles, row['NO']
    else:
        return [], None
def process_chunk_reactant(chunk_df,smile_max_score_pd):
    smiles_success = []
    success_rxndbid = []
    for result in map(partial(process_reaction_reactant,smile_max_score_pd=smile_max_score_pd), chunk_df.iterrows()):
        if result[0]:  # check whether the result is non-empty
            smiles_success.extend(result[0])
        if result[1] is not None:
            success_rxndbid.append(result[1])
    return smiles_success, success_rxndbid
def process_chunk_product(chunk_df,smile_max_score_pd):
    smiles_success = []
    success_rxndbid = []
    for result in map(partial(process_reaction_product,smile_max_score_pd=smile_max_score_pd), chunk_df.iterrows()):
        if result[0]:  # check whether the result is non-empty
            smiles_success.extend(result[0])
        if result[1] is not None:
            success_rxndbid.append(result[1])
    return smiles_success, success_rxndbid
def process_reactions_in_parallel_reactant(rxndb,origin_smile_max_score_pd, num_processes=100, num_iterations=1):
    num = 0
    tmp_smile_max_score_pd = origin_smile_max_score_pd
    while num < num_iterations:
        num+=1
        # pool = multiprocessing.Pool(num_processes)
        smiles_success = []
        success_rxndbid = []
        chunks = np.array_split(rxndb, 100)
        with multiprocessing.Pool(num_processes) as pool:
            for result in tqdm(pool.imap(partial(process_chunk_reactant,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                smiles_success.extend(result[0])
                success_rxndbid.extend(result[1])
            for result in tqdm(pool.imap(partial(process_chunk_product,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                smiles_success.extend(result[0])
                success_rxndbid.extend(result[1])
        # pool.close()
        # pool.join()
        
        smiles_success = list(set(smiles_success))
        success_rxndbid = list(set(success_rxndbid))
        # Process cumulative successful SMILES
        for smile in smiles_success:
            if get_score_from_smiles(smile, tmp_smile_max_score_pd) < 1:
                new_row = {'smile': smile, 'score': 1, 'sim_smile': 'sys', 'inchikey0': smiles2inchikey0(smile)}
                tmp_smile_max_score_pd = tmp_smile_max_score_pd._append(new_row, ignore_index=True)

        # smiles_success = [normalize_smiles(met) for met in smiles_success]
        # smiles_success = list(set(smiles_success))

        print(f'Iteration {num} - Current success count: {len(smiles_success)}')
        print(f'Iteration {num} - Current success_rxndbid count: {len(success_rxndbid)}')
        print('============================================================================')

    print('final success', len(smiles_success))
    print('final success_rxndbid', len(success_rxndbid))
    # print(success_rxndbid)
    return smiles_success, success_rxndbid,tmp_smile_max_score_pd

In [4]:
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)
success_smiles = load_pickle(not_lipid_YMDB_success_met_smile_add_no_ec_path)
lipid_success_smiles = load_pickle(lipid_YMDB_success_met_smile_file)   
lipid_success_smiles_remain = load_pickle(lipid_YMDB_success_met_smile_file_remian)
uptake_met = pd.read_csv(uptake_met_path)   
uptake_smiles = uptake_met['SMILES'].to_list()
sink_smiles = yeast8_total_smiles + success_smiles + uptake_smiles + lipid_success_smiles 
sink_smiles = list(set(sink_smiles))
sink0 = [smiles2inchikey0(x) for x in sink_smiles]
sink0 = list(set(sink0))
print(len(sink0))


13745


In [6]:
lipid_success_smiles_remain = load_pickle(lipid_YMDB_success_met_smile_file_remian)

differenct_metabolites_inchikey0 = [smiles2inchikey0(x) for x in lipid_success_smiles_remain]
rxndb_to_model_add_no_ec = pd.read_csv(rxndb_to_model_add_no_ec_path)
rxndb_to_model_add_no_ec.head()

,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
0,rxn2,NaN,MNXR164590,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...
1,rxn3,NaN,MNXR164590,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...
2,rxn4,NaN,MNXR164590,CC(C)=CCCC(C)=CCCC(C)=CC=O.O=O>>CC(C)=CCCC(C)=...,CC(C)=CCCC(C)=CCCC(C)=CC=O.O=O.NC(=O)C1=CN([C@...
3,rxn5,NaN,MNXR164590,C=C(C)C1CC=C(C=O)CC1.O=O>>C=C(C)C1CC=C(C(=O)O)...,C=C(C)C1CC=C(C=O)CC1.O=O.NC(=O)C1=CN([C@@H]2O[...
4,rxn6,NaN,MNXR164590,CCCCCCCCCCCCCCCC=O.O=O>>CCCCCCCCCCCCCCCC(=O)O.O,CCCCCCCCCCCCCCCC=O.O=O.NC(=O)C1=CN([C@@H]2O[C@...


In [7]:
len(differenct_metabolites_inchikey0)

1045

In [8]:
def find_reaction_containing_metabolite(rxndb_to_model_add_no_ec,inchikey0):
    rxndb_to_model_add_no_ec['product_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[1].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['product_inchikey0'].progress_apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]
def find_reaction_containing_metabolite_( rxndb_to_model_add_no_ec,inchikey0):
    rxndb_to_model_add_no_ec['reactant_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[0].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['reactant_inchikey0'].progress_apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]

In [22]:
# chunks = np.array_split(rxndb_to_model_add_no_ec, 10)
# with multiprocessing.Pool(10) as pool:
#     results = pool.imap(partial(find_reaction_containing_metabolite_, inchikey0=differenct_metabolites_inchikey0), chunks)
# rxndb_to_model_add_no_ec_with_target_ = pd.concat(results)
rxndb_to_model_add_no_ec_with_target = find_reaction_containing_metabolite( rxndb_to_model_add_no_ec,differenct_metabolites_inchikey0)

100%|██████████| 1655549/1655549 [01:00<00:00, 27402.76it/s]


In [24]:
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target.drop(['product_inchikey0','target_num'],axis=1)


In [25]:
rxndb_to_model_add_no_ec_with_target_ = find_reaction_containing_metabolite_( rxndb_to_model_add_no_ec,differenct_metabolites_inchikey0)


100%|██████████| 1655549/1655549 [00:42<00:00, 39269.39it/s]


In [26]:
rxndb_to_model_add_no_ec_with_target_.shape

(446033, 8)

In [27]:
rxndb_to_model_add_no_ec_with_target_ = rxndb_to_model_add_no_ec_with_target_.drop(['product_inchikey0','reactant_inchikey0','target_num'],axis=1)


In [28]:
rxndb_to_model_add_no_ec_with_target = pd.concat([rxndb_to_model_add_no_ec_with_target, rxndb_to_model_add_no_ec_with_target_])
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target.drop_duplicates(subset=['NO'])
rxndb_to_model_add_no_ec_with_target.shape

(449886, 5)

In [29]:
rxndb_to_model_add_no_ec_with_target.to_csv(rxndb_to_model_with_target_path, index=False)


In [9]:
rxndb_to_model_add_no_ec_with_target = pd.read_csv(rxndb_to_model_with_target_path)

In [26]:
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target[rxndb_to_model_add_no_ec_with_target['EC number'].isna()]
rxndb_to_model_add_no_ec_with_target.shape

(73, 5)

In [30]:
rxndb_met_max_score_pd = pd.DataFrame({'smile': sink_smiles})
rxndb_met_max_score_pd['score'] = 1
rxndb_met_max_score_pd['sim_smile'] = ''
rxndb_met_max_score_pd['inchikey0'] = rxndb_met_max_score_pd['smile'].apply(smiles2inchikey0)
rxndb_met_max_score_pd.to_csv(rxndb_met_max_score_file, index=False)

In [10]:
rxndb_met_max_score_pd = pd.read_csv(rxndb_met_max_score_file)


In [27]:
# rxndb_met_max_score_pd_top50_0_3 = pd.read_csv(rxndb_met_max_score_pd_top50_0_3_path)
# rxndb_met_max_score_pd_top50_0_3['inchikey0'] = rxndb_met_max_score_pd_top50_0_3['smile'].apply(smiles2inchikey0)
# rxndb_met_max_score_pd = pd.concat([rxndb_met_max_score_pd,rxndb_met_max_score_pd_top50_0_3])

In [11]:
# rxndb_met_max_score_pd_top50_0_3  = pd.read_csv(rxndb_met_max_score_pd_top50_0_3_path)
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_to_model_add_no_ec_with_target, rxndb_met_max_score_pd, num_processes=30, num_iterations=2)


100%|██████████| 100/100 [01:03<00:00,  1.56it/s]


Iteration 1 - Current success count: 585
Iteration 1 - Current success_rxndbid count: 4573


100%|██████████| 100/100 [01:07<00:00,  1.49it/s]


Iteration 2 - Current success count: 3041
Iteration 2 - Current success_rxndbid count: 214397
final success 3041
final success_rxndbid 214397


In [13]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_to_model_add_no_ec_with_target, smiles_max_score_tmp_pd, num_processes=40, num_iterations=2)


100%|██████████| 100/100 [01:00<00:00,  1.64it/s]


Iteration 1 - Current success count: 5116
Iteration 1 - Current success_rxndbid count: 449886


100%|██████████| 100/100 [00:53<00:00,  1.86it/s]


Iteration 2 - Current success count: 5116
Iteration 2 - Current success_rxndbid count: 449886
final success 5116
final success_rxndbid 449886


In [18]:
rxndb_to_model_success = rxndb_to_model_add_no_ec_with_target[rxndb_to_model_add_no_ec_with_target['NO'].isin(success_rxndbid)]
rxndb_to_model_success.shape

(449886, 5)

In [19]:
rxndb_to_model_success.to_csv(rxndb_to_model_with_target_success_path, index=False)

In [31]:
rxndb_to_model_success.head()

,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
0,rxn801,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
1,rxn808,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
2,rxn815,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
3,rxn1290,3.1.3;3.1.3.-,MNXR116844,CCCCCCCCCCCCCC=CC(O)C([NH3+])COP(=O)([O-])[O-]...,CCCCCCCCCCCCCC=CC(O)C([NH3+])COP(=O)([O-])[O-]...
4,rxn1297,3.1.3;3.1.3.-,MNXR116844,CCCCCCCCCCCCCC=CC(O)C(N)COP(=O)([O-])[O-].O>>C...,CCCCCCCCCCCCCC=CC(O)C(N)COP(=O)([O-])[O-].O>>C...


In [14]:
success_inchikey0 = [smiles2inchikey0(x) for x in smiles_success]
intesect = set(success_inchikey0).intersection(set(differenct_metabolites_inchikey0))
print(len(intesect))

1038


In [17]:
len(list(set(differenct_metabolites_inchikey0)))

1038